## Python notebook to clean the coverage table
- Remove unwanted strings from headers
- Separate 2019 from 2021 coverage table
- Normalize the coverage table

In [ ]:
# import 
import pandas as pd
import numpy as np


In [2]:
# open df with coverage table
df = pd.read_csv('../data/raw_coverage_table.tsv', sep='\t')

# Remove string from colnames
df.columns = df.columns.str.rstrip(' Mean')


# drop the 2019 columns
df = df[df.columns.drop(list(df.filter(regex='BB_')))]

# print
df.head()

,Contig,BB_1_1_L001,BB_1_2_L001,BB_2_1_L001,BB_2_2_L001,BB_3_1_L001,BB_4_1_L001,BB_4_2_L001,BB_5_1_L001,BB_5_2_L001,...,GPS3_T3_V,HC1_T1_V,HC1_T2_V,HC1_T3_V,HC2_T1_V,HC2_T2_V,HC2_T3_V,HC3_T1_V,HC3_T2_V,HC3_T3_V
0,Hopland_2021_Santos_V01_VIR_S446_L004_100740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Hopland_2021_Santos_V01_VIR_S446_L004_10247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Hopland_2021_Santos_V01_VIR_S446_L004_102687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Hopland_2021_Santos_V01_VIR_S446_L004_104104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Hopland_2021_Santos_V01_VIR_S446_L004_104370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# sum all abundances except the contig name
df['sum'] = df.drop('Contig', axis=1).sum(axis=1)

# Remove all where vOTU isn't present in df
df= df[df['sum'] > 0]

len(df)

14891

In [27]:
# Make a df from only contig names
df_contigs = df[['Contig']]

# drop contig names from real df
df.drop('Contig', axis=1, inplace=True) 
df.drop('sum', axis=1, inplace=True) 

In [28]:
# open the multiplication factors
df_norm = pd.read_csv('../data/norm_factors.csv')

#create dict from norm factors
norm_dict = dict(df_norm.values)

len(df_norm)

63

In [29]:
# MUltiply df with norm factors
df  = df.mul(norm_dict)

# add contig names back
df = df_contigs.join(df)

# write to csv
df.to_csv('../data/covtab_normalized.csv', index=False)

In [4]:
# make a dataframe with ones and zeros
# Make a df from only contig names
df_contigs = df[['Contig']]

# drop contig names from real df
df.drop('Contig', axis=1, inplace=True) 

# ones and 0
df[df > 0] = 1


# add contig names back
df = df_contigs.join(df)

# write to csv
df.to_csv('../data/220810_covtab_all_01.csv', index=False)

In [34]:
# see where contigs from this study are
df['own'] = df.Contig.str.count('BB_2021_')

# remove all contigs from this study to keep only the ones from POGEON
df = df[df['own'] != 1]

len(df)

565

In [18]:
# make sums for upset plot
df['BMLB'] = df.filter(like='BMLB').sum(1)
df['BMLF'] = df.filter(like='BMLF').sum(1)
df['BMLS'] = df.filter(like='BMLS').sum(1)
df['DP'] = df.filter(like='DP').sum(1)
df['GPB'] = df.filter(like='GPB').sum(1)
df['GPS'] = df.filter(like='GPS').sum(1)
df['HC'] = df.filter(like='HC').sum(1)

df['BMLB_2019'] = df.filter(like='BB_1_').sum(1)
df['GPS_2019'] = df.filter(like='BB_2_').sum(1)
df['DP_2019'] = df.filter(like='BB_3_').sum(1)
df['GRASS_2019'] = df.filter(like='BB_').sum(1)

df['GRASS_2019'] = df['GRASS_2019'] - (df['BMLB_2019'] + df['GPS_2019'] + df['DP_2019'])


# Make a df from only contig names
df_contigs = df[['Contig']]

# drop contig names from real df
df.drop('Contig', axis=1, inplace=True) 

# ones and 0
df[df > 0] = 1


# add contig names back
df = df_contigs.join(df)


In [19]:
df.to_csv('../data/220826_covtab_for_upset.csv', index=False)